# Memory, base class

This implements the Cassandra-specific I/O facilities, and is then used by LangChain to power every other, more sophisticated, abstraction around keeping chat memory.

## Colab-specific setup

Make sure you have a database available. This is a Colab notebook, so here it is assumed you will use
an Astra DB cloud instance. For usage with an Apache Cassandra® cluster, switch to a local setup
as instructed [on cassio.org](https://cassio.org/more_info/#use-a-local-vector-capable-cassandra).

Get ready to supply the connection parameters: Database ID and Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details.
Remember you need a Token with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/memory-basic/)._


In [ ]:
# install required dependencies
! pip install -q --progress-bar off \
    "langchain>=0.0.348" \
    "cassio>=0.1.3" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2" 
exit()

⚠️ **Do not mind a "Your session crashed..." message you may see.**

It was us, making sure your kernel restarts with all the correct dependency versions. _You can now proceed with the notebook._

### DB connection parameters
Example values (check your Astra dashboard):
- Database ID example: `01234567-89ab-cdef-0123-456789abcdef`
- Token example: `AstraCS:6gBhNmsk135....` (ensure it has a role of at least "Database Administrator")
- _Keyspace. Optional, string if provided. Example:_ `default_keyspace`

In [ ]:
import os
from getpass import getpass

# Input your Astra DB connection parameters and secrets:

os.environ["ASTRA_DB_ID"] = input("ASTRA_DB_ID = ")

os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

_keyspace = input("(Optional) ASTRA_DB_KEYSPACE = ")
if _keyspace:
    os.environ["ASTRA_DB_KEYSPACE"] = _keyspace

### Colab preamble completed

The following cells constitute the demo notebook proper.

# Memory, base class

This implements the Cassandra-specific I/O facilities, and is then used by LangChain to power every other, more sophisticated, abstraction around keeping chat memory.

In [1]:
from langchain.memory import CassandraChatMessageHistory

A database connection is needed. _(If on a Colab, the only supported option is the cloud service Astra DB.)_

In [2]:
# Getting ready to initialize the DB connection globally ...
import os

import cassio

Select your choice of database by editing this cell, if needed:

In [3]:
database_mode = "astra_db"  # only "astra_db" supported on Colab

In [4]:
if database_mode == "astra_db":
    cassio.init(
        database_id=os.environ["ASTRA_DB_ID"],
        token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
        keyspace=os.environ.get("ASTRA_DB_KEYSPACE"),  # this is optional
    )

In [5]:
if database_mode == "cassandra":
    # Cassandra not supported on Colab - please define your own getCassandraCQLSession/getCassandraCQLKeyspace
    cassio.init(
        session=getCassandraCQLSession(),
        keyspace=getCassandraCQLKeyspace(),
    )

## Create the ChatMessageHistory

In [6]:
message_history = CassandraChatMessageHistory(
    session_id='test-session',
    session=None,
    keyspace=None,
    ttl_seconds = 3600,
)
message_history.clear()

The memory starts empty:

In [7]:
message_history.messages

[]

## Insert and retrieve chat messages

In [8]:
message_history.add_user_message('Hello, I am human.')

In [9]:
message_history.add_ai_message('Hello, I am the bot.')

In [10]:
message_history.messages

[HumanMessage(content='Hello, I am human.', additional_kwargs={}, example=False),
 AIMessage(content='Hello, I am the bot.', additional_kwargs={}, example=False)]

In [11]:
message_history.clear()

In [12]:
message_history.messages

[]

## What now?

This demo is hosted [here](https://cassio.org/frameworks/langchain/memory-basic/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/langchain/about/)**.